<a href="https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%209/Worksheet 9-2 Stiff ODEs and Numerical Stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Worksheet 9.2: Stiff ODEs and Numerical Stability

## ENGR& 240: Engineering Computations

### Objectives
- Understand what makes an ODE system "stiff"
- Explore numerical instability with explicit methods on stiff systems
- Compare explicit vs implicit methods for stiff systems
- Investigate the effect of step size on stability and efficiency

## Introduction

Some differential equation systems are called "stiff" because they contain processes that occur on very different time scales. This creates numerical challenges for explicit methods like Euler's method and Runge-Kutta methods.

In this worksheet, we'll explore stiffness through a chemical reaction system that exhibits rapid equilibration alongside slower overall changes. You'll discover why explicit methods struggle and how implicit methods can solve these problems efficiently.

### The Chemical Reaction System

Consider a three-species chemical reaction system:

$$\frac{dA}{dt} = -k_1 A B + k_2 C$$
$$\frac{dB}{dt} = -k_1 A B + k_2 C - k_3 B C$$
$$\frac{dC}{dt} = k_1 A B - k_2 C - k_3 B C$$

Where A, B, and C are species concentrations, and $k_1$, $k_2$, $k_3$ are reaction rate constants.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import time

## Provided: RK4 System Solver

Here's a complete implementation of the fourth-order Runge-Kutta method for systems of ODEs:

In [ ]:
def odeRK4sys(dydt, tspan, y0, h, *args):
    """
    Solve system of ODEs using fourth-order Runge-Kutta method.

    Args:
        dydt (callable): Function defining the ODE system
        tspan (array-like): [t0, tf] initial and final values
        y0 (array-like): Initial values for system
        h (float): Step size
        *args: Additional parameters used by dydt

    Returns:
        tuple: (t, y)
            t: Array of time values
            y: Array of solution values (each column is one variable)
    """
    # Input validation
    if not callable(dydt):
        raise ValueError("dydt must be a callable function")
    if len(tspan) != 2:
        raise ValueError("tspan must be [t0, tf]")

    t0, tf = tspan

    # Create time array
    t = np.arange(t0, tf + h/2, h)  # h/2 term handles floating-point issues
    n = len(t)

    # Ensure tf is included
    if t[-1] < tf:
        t = np.append(t, tf)
        n += 1

    # Initialize solution array
    y0 = np.array(y0)
    n_eqn = len(y0)
    y = np.ones((n, n_eqn))
    y[0] = y0

    # Implement RK4 method
    for i in range(n-1):
        # Calculate step size for this iteration
        h_step = t[i+1] - t[i]

        # First step
        k1 = dydt(t[i], y[i], *args)

        # Second step
        ymid2 = y[i] + k1 * h_step/2
        k2 = dydt(t[i] + h_step/2, ymid2, *args)

        # Third step
        ymid3 = y[i] + k2 * h_step/2
        k3 = dydt(t[i] + h_step/2, ymid3, *args)

        # Fourth step
        yend = y[i] + k3 * h_step
        k4 = dydt(t[i] + h_step, yend, *args)

        # Combine steps
        phi = (k1 + 2*k2 + 2*k3 + k4) / 6
        y[i+1] = y[i] + phi * h_step

    return t, y

## Task 1: Implementing the Chemical Reaction System

Implement the chemical reaction system as a Python function. This function should return the derivatives for all three species.

In [ ]:
def chemical_reaction(t, y, k1, k2, k3):
    """
    Chemical reaction system with three species A, B, C.

    Args:
        t (float): Current time
        y (array): Current concentrations [A, B, C]
        k1, k2, k3 (float): Reaction rate constants

    Returns:
        array: Derivatives [dA/dt, dB/dt, dC/dt]
    """
    A, B, C = y[0], y[1], y[2]

    # TODO: Implement the reaction rate equations

    return np.array([dA_dt, dB_dt, dC_dt])

## Task 2: Discovering Stiffness - Effect of Step Size on RK4

Let's explore what happens when we use RK4 with different step sizes on this system. We'll use parameters that make the system stiff.

**Note**: Be prepared - some of these simulations might behave unexpectedly!

In [ ]:
# System parameters that create stiffness
k1 = 0.25    # Moderate forward reaction
k2 = 0.1     # Slow reverse reaction
k3 = 125     # Very fast consumption reaction - this creates stiffness!

# Initial conditions
y0 = [1.0, 2.2, 0.0]  # [A, B, C]
tspan = [0, 6]         # Time span

# Different step sizes to test
step_sizes = [0.015, 0.01, 0.005]
step_labels = ['h = 0.015', 'h = 0.01', 'h = 0.005']

# TODO: Use RK4 to solve the system with each step size
results_rk4 = []

for h in step_sizes:
    print(f"Solving with step size h = {h}...")

    # TODO: Call odeRK4sys with the chemical_reaction function
    t, y = odeRK4sys(chemical_reaction, tspan, y0, h, k1, k2, k3)

    results_rk4.append((t, y))

    # Print some diagnostics
    print(f"  Final concentrations: A={y[-1,0]:.3f}, B={y[-1,1]:.3f}, C={y[-1,2]:.3f}")
    print(f"  Max values: A={np.max(y[:,0]):.3f}, B={np.max(y[:,1]):.3f}, C={np.max(y[:,2]):.3f}")
    print(f"  Min values: A={np.min(y[:,0]):.3f}, B={np.min(y[:,1]):.3f}, C={np.min(y[:,2]):.3f}")
    print()

In [ ]:
# Create plots comparing the three step sizes
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

for i, ((t, y), label) in enumerate(zip(results_rk4, step_labels)):
    ax = axes[i]

    # Plot A, B, and scaled C concentrations
    ax.plot(t, y[:, 0], 'b-', label='A')

    # TODO: Add two more statements analogous to the one above to add plots
    # of B and scaled C concentration (plot 200*C for visibility)

    # Your code here

    ax.set_title(f'RK4 with {label}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Concentration')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Discussion Questions for Task 2

**Look at your plots and results, then answer:**

1. **What do you observe about the behavior with different step sizes?**
   - Are all solutions physically reasonable?
   - Do you see any instabilities or unrealistic behavior?

2. **Which concentrations become negative (if any)?**
   - Is this physically meaningful for chemical concentrations?

3. **What happens as you decrease the step size from 0.015 to 0.005?**
   - Does the solution stabilize?

## Task 3: Comparing with Built-in Solvers

Now let's see how Python's built-in solvers handle this same problem. We'll use several different methods, including some specifically designed for stiff systems.

In [ ]:
# Built-in solver comparison
methods = ['RK45', 'Radau', 'BDF']
method_descriptions = {
    'RK45': 'Explicit Runge-Kutta (similar to our RK4)',
    'Radau': 'Implicit Runge-Kutta (good for stiff systems)',
    'BDF': 'Backward Differentiation Formula (designed for stiff systems)'
}

# Solve using scipy's solve_ivp with different methods
results_scipy = {}
solve_times = {}

for method in methods:
    print(f"Solving with {method} ({method_descriptions[method]})...")

    # Time the solution
    start_time = time.time()

    # TODO: Use solve_ivp to solve the system
    sol = solve_ivp(chemical_reaction, tspan, y0,
                    method=method,
                    args=(k1, k2, k3),
                    rtol=1e-8, atol=1e-8,
                    dense_output=True)

    end_time = time.time()
    solve_times[method] = end_time - start_time

    # Store results
    results_scipy[method] = sol

    # Print diagnostics
    print(f"  Success: {sol.success}")
    print(f"  Function evaluations: {sol.nfev}")
    print(f"  Solution time: {solve_times[method]:.4f} seconds")
    print(f"  Final concentrations: A={sol.y[0,-1]:.6f}, B={sol.y[1,-1]:.6f}, C={sol.y[2,-1]:.6f}")
    print()

In [ ]:
# Create comparison plots for the different methods
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

# Plot results from each method
for i, method in enumerate(methods):
    ax = axes[i]
    sol = results_scipy[method]

    # Create a dense time grid for smooth plotting
    t_dense = np.linspace(tspan[0], tspan[1], 1000)
    y_dense = sol.sol(t_dense)

    # Plot the concentrations
    ax.plot(t_dense, y_dense[0], 'b-', label='A')
    ax.plot(t_dense, y_dense[1], 'r-', label='B')
    ax.plot(t_dense, 200 * y_dense[2], 'g-', label='200*C')

    ax.set_title(f'{method}\n({method_descriptions[method]})\n'
                f'Time: {solve_times[method]:.4f}s, Evals: {sol.nfev}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Concentration')
    ax.legend()
    ax.grid(True)

# Add a comparison of the best RK4 result
ax = axes[3]
t_rk4, y_rk4 = results_rk4[2]  # Use the smallest step size result
ax.plot(t_rk4, y_rk4[:, 0], 'b-', label='A')
ax.plot(t_rk4, y_rk4[:, 1], 'r-', label='B')
ax.plot(t_rk4, 200 * y_rk4[:, 2], 'g-', label='200*C')
ax.set_title(f'Our RK4 (h=0.005)\nSteps: {len(t_rk4)}')
ax.set_xlabel('Time')
ax.set_ylabel('Concentration')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

## Task 4: Efficiency Analysis

Let's investigate how efficient each method is by comparing the number of function evaluations and solution time.

In [ ]:
# Create efficiency comparison
print("=== EFFICIENCY COMPARISON ===")
print(f"{'Method':<15} {'Time (s)':<12} {'Function Evals':<15} {'Steps':<10}")
print("-" * 55)

# Built-in methods
for method in methods:
    sol = results_scipy[method]
    print(f"{method:<15} {solve_times[method]:<12.4f} {sol.nfev:<15} {len(sol.t):<10}")

# Our RK4 method
t_rk4, y_rk4 = results_rk4[2]  # Smallest step size
# For RK4, function evaluations = 4 * number of steps
rk4_evals = 4 * (len(t_rk4) - 1)
print(f"{'Our RK4':<15} {'N/A':<12} {rk4_evals:<15} {len(t_rk4):<10}")


In [ ]:
# Create bar charts comparing efficiency metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Function evaluations comparison
method_names = list(methods) + ['Our RK4']
func_evals = [results_scipy[method].nfev for method in methods] + [rk4_evals]

bars1 = ax1.bar(method_names, func_evals, color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
ax1.set_ylabel('Function Evaluations')
ax1.set_title('Computational Cost Comparison')
ax1.set_yscale('log')  # Use log scale due to large differences

# Add value labels on bars
for bar, val in zip(bars1, func_evals):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f'{val}', ha='center', va='bottom')

# Solution time comparison (excluding RK4 since we didn't time it)
times = [solve_times[method] for method in methods]
bars2 = ax2.bar(methods, times, color=['skyblue', 'lightcoral', 'lightgreen'])
ax2.set_ylabel('Solution Time (seconds)')
ax2.set_title('Solution Time Comparison')

# Add value labels on bars
for bar, val in zip(bars2, times):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f'{val:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### Discussion Questions for Tasks 3 and 4
Review the results for tasks 3 and 4 and answer the following questions.

1. Which methods required the fewest function evaluations?
2. Which methods were fastest?
3. How does our RK4 compare in terms of computational cost?

## Understanding Stiffness: Theory Behind the Observations

Now that you've discovered the challenges of stiff systems through hands-on exploration, let's understand the theory behind what you observed.

### What Makes a System "Stiff"?

A differential equation system is **stiff** when it contains processes operating on vastly different time scales. In our chemical reaction system:

- **Fast process**: The reaction $B + C \rightarrow$ products with rate constant $k_3 = 125$ (or 5000 in Task 5)
- **Slow processes**: Formation and decomposition reactions with $k_1 = 0.25$ and $k_2 = 0.1$

The ratio of fastest to slowest time scales is roughly $k_3/k_2 = 125/0.1 = 1250$ (or 50,000 in the extreme case).

### The Mathematical Foundation: Jacobian Matrix and Eigenvalues

The key to understanding stiffness lies in the **Jacobian matrix** of our ODE system. The Jacobian is NOT a property of the numerical method—it's a property of the differential equation system itself.

For our chemical reaction system:
$$\frac{d\mathbf{y}}{dt} = \mathbf{f}(t, \mathbf{y}) = \begin{bmatrix} -k_1 A B + k_2 C \\ -k_1 A B + k_2 C - k_3 B C \\ k_1 A B - k_2 C - k_3 B C \end{bmatrix}$$

The Jacobian matrix is:
$$\mathbf{J} = \frac{\partial \mathbf{f}}{\partial \mathbf{y}} = \begin{bmatrix}
-k_1 B & -k_1 A & k_2 \\
-k_1 B & -k_1 A - k_3 C & k_2 - k_3 B \\
k_1 B & k_1 A - k_3 C & -k_2 - k_3 B
\end{bmatrix}$$

**For our specific system with typical concentrations** (A ≈ 1, B ≈ 2, C ≈ 0.01):
$$\mathbf{J} \approx \begin{bmatrix}
-0.5 & -0.25 & 0.1 \\
-0.5 & -0.25 - 1.25 & 0.1 - 250 \\
0.5 & 0.25 - 1.25 & -0.1 - 250
\end{bmatrix} = \begin{bmatrix}
-0.5 & -0.25 & 0.1 \\
-0.5 & -1.5 & -249.9 \\
0.5 & -1.0 & -250.1
\end{bmatrix}$$

The **eigenvalues** of this matrix are approximately: $\lambda_1 \approx -250$, $\lambda_2 \approx -1$, $\lambda_3 \approx -0.1$

These eigenvalues represent the **characteristic time scales** of the system:
- $\lambda_1 \approx -250$: Very fast process (time scale ≈ 1/250 = 0.004 time units)
- $\lambda_2 \approx -1$: Moderate process (time scale ≈ 1 time unit)  
- $\lambda_3 \approx -0.1$: Slow process (time scale ≈ 10 time units)

The **stiffness ratio** is $|\lambda_{max}|/|\lambda_{min}| = 250/0.1 = 2500$.

### Why Explicit Methods Struggle

Every numerical method has a **stability region** in the complex plane. For the method to remain stable, all eigenvalues of the Jacobian must satisfy:

$$h \cdot \lambda_i \in \text{stability region}$$

**For RK4 and similar explicit methods:**
- Stability region is roughly a circle with radius ≈ 2.8
- This requires: $|h \cdot \lambda_{max}| < 2.8$
- **Stability limit**: $h < \frac{2.8}{|\lambda_{max}|}$

**For our system**: $h < \frac{2.8}{250} \approx 0.011$

**For the extreme system (Task 5)**: $h < \frac{2.8}{5000} \approx 0.0006$

This means:
- Even though we only care about slow, long-term behavior (time scales of 1-10 units)
- We're forced to take tiny steps (0.011 or smaller) because of the fast process
- Result: Thousands of tiny steps to solve what should be a simple problem

### Why Implicit Methods Excel

Implicit methods like **Radau** and **BDF** have fundamentally different stability properties:

**Stability regions:**
- **RK4**: Small circular region (radius ≈ 2.8)
- **Radau**: Large region including most of the left half-plane
- **BDF**: Entire left half-plane for reasonable orders

This means:
- **Implicit methods can take large steps** even when $|\lambda_{max}|$ is huge
- They're **L-stable** (stable for arbitrarily stiff problems)
- **Trade-off**: Each step requires solving nonlinear equations (more work per step)
- **Net result**: Far fewer steps needed, so much more efficient overall

### The Efficiency Mathematics

From your observations, the efficiency advantage comes from:

**Explicit methods (RK4, RK45):**
- Required step size: $h \approx \frac{2.8}{|\lambda_{max}|}$
- Number of steps: $N \approx \frac{T_{final}}{h} = \frac{T_{final} \cdot |\lambda_{max}|}{2.8}$
- Function evaluations: $4N$ (for RK4) or similar

**Implicit methods (Radau, BDF):**
- Can use step sizes based on **accuracy requirements**, not stability
- Typically $h \approx 0.1$ to $1.0$ (much larger!)
- Far fewer steps needed
- Each step costs more, but total cost is much less

### Real-World Impact

This explains why:
1. **Stiff ODE solvers were invented** - explicit methods simply can't handle many real problems
2. **Method selection matters enormously** - wrong choice can make problems unsolvable
3. **Implicit methods dominate** in applications like:
   - Chemical kinetics (like our example)
   - Circuit simulation
   - Atmospheric modeling
   - Any system with multiple time scales

The factor of 100-1000x efficiency difference you observed isn't unusual—it's typical for stiff systems!

## Task 5: Extreme Stiffness Demonstration

Let's push the stiffness to an extreme level to see even more dramatic differences between explicit and implicit methods. We'll increase k3 to make the system extremely stiff.

### Your Choice: Select a Fourth Solver for Extreme Stiffness Testing

Before proceeding to Task 5, choose one additional solver from the options below to test on the extremely stiff system. Each solver has different characteristics that will demonstrate important concepts about numerical methods.

#### Solver Options:

**1. LSODA - Automatic Stiff/Non-stiff Detection**
- Automatically detects whether the problem is stiff or non-stiff
- Switches between explicit and implicit methods as needed
- Widely used in scientific computing for "general purpose" problems
- [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) (method='LSODA')

**2. DOP853 - High-Order Explicit Method**  
- 8th-order explicit Runge-Kutta method with very high precision
- Excellent for smooth, non-stiff problems requiring high accuracy
- Will demonstrate what happens when high-order explicit methods meet stiff systems
- [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) (method='DOP853')

#### Your Task:
1. **Choose one method** from the options above based on what interests you most
2. **Research briefly** using the documentation links to understand your chosen method
3. **Modify the Task 5 code** by adding your chosen method to the `methods_extreme` list
4. **Compare your results** with classmates who chose different methods

#### Questions to Consider:
- Why might your chosen method perform well or poorly on extremely stiff systems?
- What trade-offs does your method make (accuracy vs. speed, simplicity vs. robustness)?
- When would you recommend using your chosen method in real applications?

**Modification Instructions for Task 5:**
- Change `methods_extreme = ['RK45', 'Radau', 'BDF']`
- To: `methods_extreme = ['RK45', 'Radau', 'BDF', 'YOUR_CHOSEN_METHOD']`
- The plotting and analysis code will automatically include your fourth method

In [ ]:
# Even more extreme stiffness parameters
k1_extreme = 0.25
k2_extreme = 0.1
k3_extreme = 5000  # Much larger! Creates extreme stiffness

print(f"New stiffness ratio: k3/k2 = {k3_extreme/k2_extreme}")
print("This system has processes occurring on time scales differing by 50,000x!\n")

# Same initial conditions
y0_extreme = [1.0, 2.2, 0.0]
tspan_extreme = [0, 2]  # Shorter time span to avoid extremely long computation

# Compare the three methods again
methods_extreme = ['RK45', 'Radau', 'BDF']
results_extreme = {}
times_extreme = {}

for method in methods_extreme:
    print(f"Solving extremely stiff system with {method}...")

    # Time the solution
    start_time = time.time()

    try:
        # TODO: Solve the extremely stiff system
        sol = solve_ivp(chemical_reaction, tspan_extreme, y0_extreme,
                        method=method,
                        args=(k1_extreme, k2_extreme, k3_extreme),
                        rtol=1e-6, atol=1e-9)  # Slightly relaxed tolerance for speed

        end_time = time.time()
        times_extreme[method] = end_time - start_time
        results_extreme[method] = sol

        print(f"  Success: {sol.success}")
        print(f"  Function evaluations: {sol.nfev}")
        print(f"  Solution time: {times_extreme[method]:.4f} seconds")
        print(f"  Number of time steps: {len(sol.t)}")

        if sol.success:
            print(f"  Final concentrations: A={sol.y[0,-1]:.6f}, B={sol.y[1,-1]:.6f}, C={sol.y[2,-1]:.6f}")
        print()

    except Exception as e:
        print(f"  ERROR: {str(e)}")
        times_extreme[method] = float('inf')
        print()

### Plotting Extreme Stiffness Results

In [ ]:
# Plot results from the extremely stiff system
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, method in enumerate(methods_extreme):
    ax = axes[i]

    if method in results_extreme and results_extreme[method].success:
        sol = results_extreme[method]

        # Use the actual solution points instead of dense output
        t_plot = sol.t
        y_plot = sol.y

        # Plot the concentrations
        ax.plot(t_plot, y_plot[0], 'b-', label='A', linewidth=2)
        ax.plot(t_plot, y_plot[1], 'r-', label='B', linewidth=2)
        ax.plot(t_plot, 10000 * y_plot[2], 'g-', label='10000*C', linewidth=2)

        ax.set_title(f'{method} - Extremely Stiff System\n'
                    f'Time: {times_extreme[method]:.4f}s\n'
                    f'Function Evals: {sol.nfev}')
    else:
        ax.text(0.5, 0.5, f'{method}\nFailed or\nToo Slow',
               ha='center', va='center', transform=ax.transAxes, fontsize=16)
        ax.set_title(f'{method} - Extremely Stiff System')

    ax.set_xlabel('Time')
    ax.set_ylabel('Concentration')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Extreme Stiffness Efficiency Comparison

In [ ]:
# Create dramatic efficiency comparison
print("=== EXTREME STIFFNESS EFFICIENCY COMPARISON ===")
print(f"{'Method':<15} {'Time (s)':<12} {'Function Evals':<15} {'Success':<10}")
print("-" * 55)

for method in methods_extreme:
    if method in results_extreme and results_extreme[method].success:
        sol = results_extreme[method]
        print(f"{method:<15} {times_extreme[method]:<12.4f} {sol.nfev:<15} {'Yes':<10}")
    else:
        print(f"{method:<15} {times_extreme.get(method, 'N/A'):<12} {'N/A':<15} {'No':<10}")

# Calculate and display efficiency ratios if multiple methods succeeded
successful_methods = [m for m in methods_extreme if m in results_extreme and results_extreme[m].success]
if len(successful_methods) > 1:
    print("\n=== EFFICIENCY RATIOS ===")
    rk45_evals = results_extreme['RK45'].nfev if 'RK45' in results_extreme and results_extreme['RK45'].success else None

    for method in successful_methods:
        if method != 'RK45' and rk45_evals:
            ratio = rk45_evals / results_extreme[method].nfev
            print(f"{method} is {ratio:.1f}x more efficient than RK45 (in function evaluations)")

## Summary Questions

1. **What is stiffness in differential equations?**
   - How did you recognize it in this worksheet?

2. **Why do explicit methods like RK4 struggle with stiff systems?**
   - What happens when you try to use larger step sizes?

3. **What are the key advantages of implicit methods for stiff systems?**
   - How much more efficient were they in your results?

4. **When would you choose an implicit method over an explicit method?**
   - What are the trade-offs to consider?